# Algorithm Training and Testing

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.utils import resample

In [ ]:
# Initialize variables
random_state = 6
test_size = 0.2
db_file = 'data/pretb_cat_final.csv'
split_date = '2020-01-01'

In [ ]:
# Load data into data frame
df = pd.read_csv(db_file,
                 encoding="utf-8",
                 sep=";",
                 header=0
                )

In [ ]:
# Include only most recent tumor boards to reflect current tumorboard guidelines
df['pretb_date'] = pd.to_datetime(df['pretb_date'], dayfirst=True)
df = df.loc[df['pretb_date'] > split_date]

In [ ]:
df.info()

In [ ]:
ord_enc = OrdinalEncoder()

In [ ]:
# Convert categorical variables from text to numerical
df['dre'] = ord_enc.fit_transform(df[['dre']])
df['site'] = ord_enc.fit_transform(df[['site']])

In [ ]:
# Define features and outcomes
feature_cols = ['age', "psa", "dre", 'site', "isup", "cylinder_pos", "cylinder_total", 'ht', 'dm', 'cad', 'bmi', 'preop']
outcome_cols = ["psma", 'conv_staging', "as", 'rp_rt']
X = df[feature_cols]
y = df[outcome_cols]

In [ ]:
# Stratification of train test split according to outcomes with lowest numbers
y_strat = y.loc[:, ['psma', 'as']]

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y_strat, random_state=random_state)

In [ ]:
# Save train and test data to csv for descriptive statistics
X_train.to_csv('data/X_train.csv', index=False)
X_test.to_csv('data/X_test.csv', index=False)
y_train.to_csv('data/y_train.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)

In [ ]:
# Impute missing values for continuous variables using median of training set
imp1 = SimpleImputer(missing_values=np.nan, strategy='median')

# Fit and transform on training set
X_train['psa'] = imp1.fit_transform(X_train[['psa']])
X_train['cylinder_pos'] = imp1.fit_transform(X_train[['cylinder_pos']])
X_train['cylinder_total'] = imp1.fit_transform(X_train[['cylinder_total']])

# Transform on test set
X_test['psa'] = imp1.fit_transform(X_test[['psa']])
X_test['cylinder_pos'] = imp1.fit_transform(X_test[['cylinder_pos']])
X_test['cylinder_total'] = imp1.fit_transform(X_test[['cylinder_total']])

In [ ]:
# Impute missing values for categorial variables using most frequent in training set
imp2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

# Fit and transform on training set
X_train['isup'] = imp2.fit_transform(X_train[['isup']])
X_train['site'] = imp2.fit_transform(X_train[['site']])
X_train['dre'] = imp2.fit_transform(X_train[['dre']])

# Transform on test set
X_test['isup'] = imp2.fit_transform(X_test[['isup']])
X_test['site'] = imp2.fit_transform(X_test[['site']])
X_test['dre'] = imp2.fit_transform(X_test[['dre']])

In [ ]:
# Train DecisionTree model
dt_clf = OneVsRestClassifier(DecisionTreeClassifier(max_leaf_nodes=50, criterion="entropy", random_state=random_state))
dt_clf.fit(X_train.to_numpy(), y_train.to_numpy())
y_pred = dt_clf.predict(X_test.to_numpy())

In [ ]:
multilabel_confusion_matrix(y_test, y_pred)

In [ ]:
print ("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred, target_names=outcome_cols))

In [ ]:
# Train RandomForest model
rnd_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, criterion="entropy", random_state=random_state))

rnd_clf.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = rnd_clf.predict(X_test.to_numpy())

In [ ]:
multilabel_confusion_matrix(y_test, y_pred)

In [ ]:
print ("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred, target_names=outcome_cols))

In [ ]:
# Train KNN model
knn_clf = OneVsRestClassifier(KNeighborsClassifier())

knn_clf.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = knn_clf.predict(X_test.to_numpy())

In [ ]:
multilabel_confusion_matrix(y_test, y_pred)

In [ ]:
print ("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred, target_names=outcome_cols))

In [ ]:
# Create test sample for model out evaluation
test_sample = X_test.iloc[13, :].to_frame().transpose()
test_output = rnd_clf.predict(test_sample)

In [ ]:
def convert_output(output):
    psma_string = "Bevorzugt PSMA-PET-CT. Alternativ Ganzkörperknochenszintigraphie und CT-Abdomen und Becken. "
    ct_scintigraphy_string = "Ganzkörperknochenszintigraphie und CT-Abdomen und Becken."
    as_string = "Der Patient erfüllt die Kriterien für eine aktive Überwachung. Falls eine definitive Therapie erwünscht ist, Angebot einer radikalen Prostatektomie oder alternativ perkutane Strahlentherapie."
    rp_rt_string = "Indikation zur radikalen Prostatektomie. Alternativ perkutane Strahlentherapie."
    
    output_string = ""
    
    if output[0] == 1:
        # PSMA
        output_string += psma_string

    if output[1] == 1:
        # CT and bone scintigraphy
        if output[0] == 0:
            output_string += ct_scintigraphy_string
            
    if output[2] == 1:
        # Active Surveillance
        output_string += as_string

    if output[3] == 1:
        # Radical prostatectomy and radiation therapy
        if output[2] == 0:
            output_string += rp_rt_string
    
    return output_string

In [ ]:
test_output = test_output[0]
output_string = convert_output(test_output)

In [ ]:
print(test_output)
print(output_string)

# Statistics

In [ ]:
from tqdm import tqdm

In [ ]:

def bootstrap(X, y, model, metric, n_iterations):
    accuracy_list = []
    for i in tqdm(range(n_iterations)):
        X_bs, y_bs = resample(X, y, replace=True)
        # make predictions
        y_hat = model.predict(X_bs)
        # evaluate model
        accuracy = metric(y_bs, y_hat)
        accuracy_list.append(accuracy)
    return accuracy_list

In [ ]:
def bootstrap_by_cat(X, y, model, metric, index, n_iterations):
    metric_list = []
    for i in tqdm(range(n_iterations)):
        X_bs, y_bs = resample(X, y, replace=True)
        # make predictions
        y_hat = pd.DataFrame(model.predict(X_bs))
        # evaluate model
        metric_value = metric(y_bs.iloc[:, index], y_hat.iloc[:, index])
        metric_list.append(metric_value)
    return metric_list

In [ ]:
def get_ci(metric):
    ci_metrics = []
    # get median
    median = np.percentile(metric, 50)

    # get 95% confidence interval
    alpha = 100-95
    lower_ci = np.percentile(metric, alpha/2)
    upper_ci = np.percentile(metric, 100-alpha/2)

    ci_metrics = [median, lower_ci, upper_ci]
    return ci_metrics

In [ ]:
def show_results(X, y, model, n_iterations):
    # Calculate metrics with bootstrapping
    accuracy = bootstrap(X, y, model, accuracy_score, n_iterations)

    precision_psma = bootstrap_by_cat(X, y, model, precision_score, 0, n_iterations)
    precision_conv_staging = bootstrap_by_cat(X, y, model, precision_score, 1, n_iterations)
    precision_as = bootstrap_by_cat(X, y, model, precision_score, 2, n_iterations)
    precision_rp_rt = bootstrap_by_cat(X, y, model, precision_score, 3, n_iterations)

    recall_psma = bootstrap_by_cat(X, y, model, recall_score, 0, n_iterations)
    recall_conv_staging = bootstrap_by_cat(X, y, model, recall_score, 1, n_iterations)
    recall_as = bootstrap_by_cat(X, y, model, recall_score, 2, n_iterations)
    recall_rp_rt = bootstrap_by_cat(X, y, model, recall_score, 3, n_iterations)

    f1_psma = bootstrap_by_cat(X, y, model, f1_score, 0, n_iterations)
    f1_conv_staging = bootstrap_by_cat(X, y, model, f1_score, 1, n_iterations)
    f1_as = bootstrap_by_cat(X, y, model, f1_score, 2, n_iterations)
    f1_rp_rt = bootstrap_by_cat(X, y, model, f1_score, 3, n_iterations)

    # Prepare results for table
    result_columns = ['precision (median)', 'precision (lower 95% CI)', 'precision (upper 95% CI)',
                    'recall (median)', 'recall (lower 95% CI)', 'recall (upper 95% CI)',
                    'f1_score (median)', 'f1_score (lower 95% CI)', 'f1_score (upper 95% CI)'          
                    ]

    index_rows = ['PSMA', 'conventional imaging', 'active surveillance', 'radical therapy']

    psma_results = np.array([get_ci(precision_psma), get_ci(recall_psma), get_ci(f1_psma)]).ravel()
    conv_staging_results = np.array([get_ci(precision_conv_staging), get_ci(recall_conv_staging), get_ci(f1_conv_staging)]).ravel()
    as_results = np.array([get_ci(precision_as), get_ci(recall_as), get_ci(f1_as)]).ravel()
    rp_rt_results = np.array([get_ci(precision_rp_rt), get_ci(recall_rp_rt), get_ci(f1_rp_rt)]).ravel()

    # Create and show results table
    result_table = pd.DataFrame([psma_results, conv_staging_results, as_results, rp_rt_results], columns=result_columns, index=index_rows)
    return result_table

    

In [ ]:
dt_acc = bootstrap(X_test.values, y_test.values, dt_clf, accuracy_score, 1000)
dt_acc_ci = get_ci(dt_acc)

In [ ]:
print('Decision Tree Classifier Accuracy:')
pd.DataFrame(np.array(dt_acc_ci).reshape(1,3), columns=['median', 'lower 95% CI', 'upper 95% CI'])

In [ ]:
# Show results for decision tree classifier
show_results(X_test, y_test, dt_clf, 1000)

In [ ]:
rnd_acc = bootstrap(X_test, y_test, rnd_clf, accuracy_score, 1000)
rnd_acc_ci = get_ci(rnd_acc)

In [ ]:
print('Random Forest Classifier Accuracy:')
pd.DataFrame(np.array(rnd_acc_ci).reshape(1,3), columns=['median', 'lower 95% CI', 'upper 95% CI'])

In [ ]:
show_results(X_test, y_test, rnd_clf, 1000)

In [ ]:
knn_acc = bootstrap(X_test, y_test, knn_clf, accuracy_score, 1000)
knn_acc_ci = get_ci(knn_acc)

In [ ]:
print('K-Nearest Neighbour Classifier Accuracy:')
pd.DataFrame(np.array(knn_acc_ci).reshape(1,3), columns=['median', 'lower 95% CI', 'upper 95% CI'])

In [ ]:
show_results(X_test, y_test, knn_clf, 1000)

# Visualizations

In [ ]:
def plot_precision_recall_curve(model, model_name, X_test, y_test):
    y_score = model.predict_proba(X_test.to_numpy())
    
    # Calculate precision-recall curve
    precision = dict()
    recall = dict()
    average_precision = dict()
    classes = ['PSMA-PET', 'Conventional imaging', 'Active Surveillance', 'Local therapy']
    n_classes = y_test.shape[1]

    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test.to_numpy()[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_test.to_numpy()[:, i], y_score[:, i])

    # Plot precision-recall curve
    plt.figure()
    colors = ['blue', 'red', 'green', 'orange']
    for i, color in zip(range(n_classes), colors):
        plt.plot(recall[i], precision[i], color=color, lw=2, linestyle='--',
                label='{0} (average precision = {1:0.2f})'
                ''.format(classes[i], average_precision[i]))

    plt.grid(True)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve for {}'.format(model_name))
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plot_precision_recall_curve(dt_clf, 'Decision Tree', X_test, y_test)

In [ ]:
plot_precision_recall_curve(rnd_clf, 'Random Forest', X_test, y_test)

In [ ]:
plot_precision_recall_curve(knn_clf, 'K-Nearest Neighbour', X_test, y_test)

In [ ]:
feature_names =['Age',
                 'PSA',
                 'DRE',
                 'Site',
                 'ISUP\ncategory',
                 'Cylinder\npositive',
                 'Cylinder\ntotal',
                 'Hypertension',
                 'Diabetes\nmellitus',
                 'CAD',
                 'BMI',
                 'Preop'
                 ]

label_names = ['PSMA-PET',
               'Conventional imaging',
               'Active Surveillance',
               'Local therapy'
               ]

In [ ]:
# Extract feature importances for each label
for i, estimator in enumerate(rnd_clf.estimators_):
# Extract feature importances for each label and store in a DataFrame
    importances = []

    for i, estimator in enumerate(rnd_clf.estimators_):
        importances.append(estimator.feature_importances_)

    # Convert to DataFrame
    importances_df = pd.DataFrame(importances, columns=feature_names, index=label_names)


In [ ]:
# Display the DataFrame
importances_df

In [ ]:
# Assuming clf is your trained OneVsRestClassifier
n_labels = len(rnd_clf.estimators_)
n_features = X.shape[1]

# Create a figure and a set of subplots
fig, axes = plt.subplots(n_labels, 1, figsize=(10, n_labels * 5))

# Plot feature importances for each label
for i, estimator in enumerate(rnd_clf.estimators_):
    axes[i].bar(range(n_features), estimator.feature_importances_)
    axes[i].set_title(f'Feature importances for {label_names[i]}')
    axes[i].set_xlabel('Features')
    axes[i].set_ylabel('Importance')
    axes[i].set_xticks(range(n_features))
    axes[i].set_xticklabels(feature_names, rotation=90)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Train DecisionTree model for demo
dt_clf_demo = DecisionTreeClassifier(max_depth=3, criterion="entropy", random_state=random_state)
dt_clf_demo.fit(X_train, y_train)

In [ ]:
from sklearn import tree

plt.figure(figsize=(15,10))
_ = tree.plot_tree(dt_clf_demo, feature_names=feature_cols, filled=True)